### Se nos brinda un nuevo dataset con el casting y crew de las peliculas.

 tendremos que tratarlas para agregarla a nuestro dataframe

In [114]:
import pandas as pd

Leemos el dataframe

In [58]:
df = pd.read_csv('data/credits.csv', low_memory=False)
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


Podemos ver que no posee datos nulos.

In [115]:
#dos variables para utilizar luego.
rango_df = df.shape[0]
indice_df = list(df.index.values)

Creamos una funcion para desanidar las columnas

In [116]:
def desanidar(indice_columna, nombre_columna, nombre_dicc):
    """
    devuelve un dataframe de un diccionario dentro de una lista
    en columnas de un dataframe.

    parametros
    ----------
    indice_columna: Indice de la columna a modificar, tipo int.
    nombre_columna: Nombre del atributo que queremos extraer de la columna
    nombre_dicc: nombre que poseera la columna con los datos extraidos.
    """
    list_values = []
    for i in range(0, rango_df):
        row = df.iloc[i,indice_columna]
        row = eval(row)
        row_values = []
        for dicc in row:
            row_values.append(dicc[nombre_columna])
        list_values.append(row_values)
    dicc = {nombre_dicc: list_values}
    df_prueba = pd.DataFrame(dicc)
    return df_prueba


Lo aplicamos a los nombres de los actores, a los trabajos y a los trabajadores de la crew

In [62]:
df_actors = desanidar(0, 'name', 'actors_names')
df_job = desanidar(1, 'job', 'jobs')
df_crew = desanidar(1, 'name', 'names')

juntamos los trabajos y el personal en un solo df

In [63]:

df_job['crew'] = df_crew
df_job.head(1)

,jobs,crew
0,"[Director, Screenplay, Screenplay, Screenplay,...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."


Creemos una pequenia funcion para extraer un dataframe de los directores.

In [64]:

directors_name = []
for i in range(0, rango_df):
    list_job = df_job.iloc[i, 0]
    list_names = df_job.iloc[i, 1]
    row_names = []
    for valor in list_job:
        #si el valor es director
        if valor == 'Director':
            #recuperamos el indice
            indice = list_job.index(valor)
            #agregamos todos los directores de esta fila a una lista
            row_names.append(list_names[indice])
        #agregamos la lista de la fila a la lista general
    directors_name.append(row_names)
dir_dict = {'director': directors_name}
df_dirs = pd.DataFrame(dir_dict)


In [65]:
df_dirs

,director
0,[John Lasseter]
1,[Joe Johnston]
2,[Howard Deutch]
3,[Forest Whitaker]
4,[Charles Shyer]
...,...
45471,[Hamid Nematollah]
45472,[Lav Diaz]
45473,[Mark L. Lester]
45474,[Yakov Protazanov]


Veamos si efectivamente hay un solo director por pelicula

In [66]:
[valor for valor in df_dirs['director'] if len(valor) > 1]

[['Allison Anders', 'Allison Anders', 'Allison Anders', 'Allison Anders'],
 ['Jean-Pierre Jeunet', 'Jean-Pierre Jeunet'],
 ['Albert Hughes', 'Albert Hughes'],
 ['Mike Gabriel', 'Mike Gabriel'],
 ['Paul Auster', 'Paul Auster'],
 ['Wayne Wang', 'Wayne Wang'],
 ['Peter Farrelly', 'Peter Farrelly'],
 ['Kevin Lima', 'Kevin Lima'],
 ['Scott McGehee', 'Scott McGehee'],
 ['Tomás Gutiérrez Alea', 'Tomás Gutiérrez Alea'],
 ['Kevin Dowling', 'Kevin Dowling'],
 ['Roger Allers', 'Roger Allers'],
 ['Robert Lee King', 'Robert Lee King', 'Robert Lee King'],
 ['Jack Baxter', 'Jack Baxter'],
 ['Simon Wincer', 'Simon Wincer', 'Simon Wincer'],
 ['Albert Hughes', 'Albert Hughes'],
 ['Annabel Jankel', 'Annabel Jankel'],
 ['D.A. Pennebaker', 'D.A. Pennebaker'],
 ['Joe Johnston', 'Joe Johnston', 'Joe Johnston'],
 ['Rob Epstein', 'Rob Epstein'],
 ['Ron Clements', 'Ron Clements'],
 ['Gary Trousdale', 'Gary Trousdale'],
 ['Hamilton Luske', 'Hamilton Luske'],
 ['Kevin Yagher', 'Kevin Yagher'],
 ['Paul Sabella', '

Podemos ver que si bien hay mas de un elemento en las filas, el director se repite, por lo que dejaremos el primero de cada uno.

In [117]:
#chequeamos por valores sorpresa
df_dirs.director.describe()

count     45476
unique    17573
top         N/D
freq        887
Name: director, dtype: object

In [68]:
lista_unicos = []
for i in range(0, rango_df):
    fila = df_dirs.iloc[i, 0]
    if fila == []:
        lista_unicos.append('N/D')
    else:
        lista_unicos.append(fila[0])

dicc_unicos = {'director': lista_unicos}
df_dirs = pd.DataFrame(dicc_unicos)
df_dirs


,director
0,John Lasseter
1,Joe Johnston
2,Howard Deutch
3,Forest Whitaker
4,Charles Shyer
...,...
45471,Hamid Nematollah
45472,Lav Diaz
45473,Mark L. Lester
45474,Yakov Protazanov


Ahora podemos concatenar nuestros actores y directores en un solo dataframe. lo hagamos.

In [110]:
df_final = pd.concat([df_dirs, df_actors], axis=1)
df_final['id'] = df['id']
df_final.id = df_final.id.apply(lambda x: str(x))
type(df_final.id.values[0])

str

Traemos el dataset de movies para hacer un merge con nuestro df.

In [118]:
data_movies = pd.read_csv('data/movies_dataset.csv', low_memory=False)


Hacemos el Merge

In [111]:
data_final = pd.merge(data_movies, df_final, on='id', how='outer', indicator=True)
data_final

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,actors_names,_merge
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,John Lasseter,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",both
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Joe Johnston,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",both
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Howard Deutch,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",both
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Forest Whitaker,"[Whitney Houston, Angela Bassett, Loretta Devi...",both
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Charles Shyer,"[Steve Martin, Diane Keaton, Martin Short, Kim...",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45537,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,Hamid Nematollah,"[Leila Hatami, Kourosh Tahami, Elham Korda]",both
45538,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,Lav Diaz,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",both
45539,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,Mark L. Lester,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",both
45540,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,[],Released,NaN,Satan Triumphant,False,0.0,0.0,Yakov Protazanov,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",both


Guardamos la data en un nuevo csv.

In [113]:
data_final.to_csv('data/movies_and_credits.csv')